# IBM capstone2

## In this project, we try to help our clients  choose a location to set up new pizza restaurant in the city of Toronto.  Since we aim to maximize our income, we focus the region of Central Toronto


## DATA And Methodology
### We use the database from foursquare to analysis our porblem. To analysis this issue, we will use K-Means for clustering

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
!pip install geopy 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!pip install folium
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [27]:
## set up account to connect Foursquare API
CLIENT_ID = 'XXZ4ET0T2B0XK2G25I5GRI32SC1B512J5FKOYDCP5IFGJ4PC' # your Foursquare ID
CLIENT_SECRET = 'ZEVV5MGYRQADZQXW440KOBJLGZAFMKBTLFYOVSUJXFDUO4VC' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XXZ4ET0T2B0XK2G25I5GRI32SC1B512J5FKOYDCP5IFGJ4PC
CLIENT_SECRET:ZEVV5MGYRQADZQXW440KOBJLGZAFMKBTLFYOVSUJXFDUO4VC


In [111]:

## We focus on Central Toronto to explore the city centre
address = 'Central Toronto'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.653963 -79.387207


In [112]:
## try to search compatitors which sells Pizza in  range of 2500 
search_query = 'Pizza'
radius = 2500
print(search_query + ' .... OK!')

Pizza .... OK!


In [113]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=XXZ4ET0T2B0XK2G25I5GRI32SC1B512J5FKOYDCP5IFGJ4PC&client_secret=ZEVV5MGYRQADZQXW440KOBJLGZAFMKBTLFYOVSUJXFDUO4VC&ll=43.653963,-79.387207&v=20180604&query=Pizza&radius=2500&limit=100'

In [114]:
results = requests.get(url).json()

In [115]:
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.shape
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood
0,4af23cabf964a520d4e621e3,Pizza Rustica Restaurant & Bar,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1581549434,False,37 Blue Jays Way,43.644919,-79.391844,"[{'label': 'display', 'lat': 43.64491920354413...",1073,M5V 3P5,CA,Toronto,ON,Canada,"[37 Blue Jays Way, Toronto ON M5V 3P5, Canada]",NaN,NaN
1,4b2438f6f964a520126424e3,Pizza Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1581549434,False,260 CHURCH STREET,43.656513,-79.377242,"[{'label': 'display', 'lat': 43.65651263631174...",851,M5B 1Z2,CA,Toronto,ON,Canada,"[260 CHURCH STREET, Toronto ON M5B 1Z2, Canada]",NaN,NaN
2,4dde735f7d8bb03c06b3db73,Boston Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1581549434,False,250 Front St,43.644059,-79.388815,"[{'label': 'display', 'lat': 43.64405883528329...",1110,M5V 3G5,CA,Toronto,ON,Canada,"[250 Front St (John St), Toronto ON M5V 3G5, C...",John St,NaN
3,4b7f161bf964a5206e1530e3,Pizza Nova,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1581549434,False,371 Front Street W,43.643272,-79.391836,"[{'label': 'display', 'lat': 43.64327168873745...",1247,M5V 3S8,CA,Toronto,ON,Canada,"[371 Front Street W (at Blue Jay Way), Toronto...",at Blue Jay Way,NaN
4,4adc9f99f964a520092e21e3,Mamma's Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1581549434,False,405 Richmond St. W,43.647943,-79.395647,"[{'label': 'display', 'lat': 43.64794272203376...",954,NaN,CA,Toronto,ON,Canada,"[405 Richmond St. W (at Spadina Ave.), Toronto...",at Spadina Ave.,NaN


In [116]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Pizza Rustica Restaurant & Bar,Pizza Place,37 Blue Jays Way,43.644919,-79.391844,"[{'label': 'display', 'lat': 43.64491920354413...",1073,M5V 3P5,CA,Toronto,ON,Canada,"[37 Blue Jays Way, Toronto ON M5V 3P5, Canada]",NaN,NaN,4af23cabf964a520d4e621e3
1,Pizza Pizza,Pizza Place,260 CHURCH STREET,43.656513,-79.377242,"[{'label': 'display', 'lat': 43.65651263631174...",851,M5B 1Z2,CA,Toronto,ON,Canada,"[260 CHURCH STREET, Toronto ON M5B 1Z2, Canada]",NaN,NaN,4b2438f6f964a520126424e3
2,Boston Pizza,Pizza Place,250 Front St,43.644059,-79.388815,"[{'label': 'display', 'lat': 43.64405883528329...",1110,M5V 3G5,CA,Toronto,ON,Canada,"[250 Front St (John St), Toronto ON M5V 3G5, C...",John St,NaN,4dde735f7d8bb03c06b3db73
3,Pizza Nova,Pizza Place,371 Front Street W,43.643272,-79.391836,"[{'label': 'display', 'lat': 43.64327168873745...",1247,M5V 3S8,CA,Toronto,ON,Canada,"[371 Front Street W (at Blue Jay Way), Toronto...",at Blue Jay Way,NaN,4b7f161bf964a5206e1530e3
4,Mamma's Pizza,Pizza Place,405 Richmond St. W,43.647943,-79.395647,"[{'label': 'display', 'lat': 43.64794272203376...",954,NaN,CA,Toronto,ON,Canada,"[405 Richmond St. W (at Spadina Ave.), Toronto...",at Spadina Ave.,NaN,4adc9f99f964a520092e21e3
5,Mamma's Pizza,Pizza Place,127 Yonge St,43.650891,-79.378632,"[{'label': 'display', 'lat': 43.650891, 'lng':...",770,M5C 1W4,CA,Toronto,ON,Canada,"[127 Yonge St, Toronto ON M5C 1W4, Canada]",NaN,NaN,552ff1d1498e5f41b0ccb3bd
6,Amato Pizza,Pizza Place,429A Yonge St,43.660215,-79.382571,"[{'label': 'display', 'lat': 43.66021482917061...",789,NaN,CA,Toronto,ON,Canada,"[429A Yonge St (at College St), Toronto ON, Ca...",at College St,NaN,4af5d885f964a520b2fd21e3
7,Pizza 2 Go,Pizza Place,65 Front St. W,43.646090,-79.379776,"[{'label': 'display', 'lat': 43.64608978691595...",1061,M5J,CA,Toronto,ON,Canada,"[65 Front St. W (Union Station), Toronto ON M5...",Union Station,NaN,4ca62112f47ea14380845d21
8,Papa Ceo Fine Italian Foods & Pizza,Pizza Place,654 Spadina Avenue,43.663300,-79.402317,"[{'label': 'display', 'lat': 43.66329982959619...",1600,M5S 2H7,CA,Toronto,ON,Canada,"[654 Spadina Avenue, Toronto ON M5S 2H7, Canada]",NaN,NaN,4ad4c05ff964a52014f720e3
9,Pizza Pizza,Pizza Place,655 Queen St W,43.647116,-79.404152,"[{'label': 'display', 'lat': 43.64711589206853...",1563,M6J 1E8,CA,Toronto,ON,Canada,"[655 Queen St W, Toronto ON M6J 1E8, Canada]",NaN,NaN,4bc3a04cabf49521650bc493


In [117]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around city centre

# add a red circle marker to represent the City Centre
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the compatitors as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

## Next, we explore the venues in city centre.

In [35]:

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=XXZ4ET0T2B0XK2G25I5GRI32SC1B512J5FKOYDCP5IFGJ4PC&client_secret=ZEVV5MGYRQADZQXW440KOBJLGZAFMKBTLFYOVSUJXFDUO4VC&ll=43.653963,-79.387207&v=20180604&radius=2500&limit=100'

In [36]:
import requests
results = requests.get(url).json()
'There are {} around City centre.'.format(len(results['response']['groups'][0]['items']))

'There are 100 around City centre.'

In [39]:
items = results['response']['groups'][0]['items']
df= json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in df.columns if col.startswith('venue.location.')] + ['venue.id']
df_filtered = df.loc[:, filtered_columns]

# filter the category for each row
df_filtered['venue.categories'] = df_filtered.apply(get_category_type, axis=1)

# clean columns
df_filtered.columns = [col.split('.')[-1] for col in df_filtered.columns]

df_filtered.head(10)

,name,categories,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,address,crossStreet,postalCode,neighborhood,id
0,Downtown Toronto,Neighborhood,43.653232,-79.385296,"[{'label': 'display', 'lat': 43.65323167517444...",174,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,NaN,NaN,NaN,5227bb01498e17bf485e6202
1,Four Seasons Centre for the Performing Arts,Concert Hall,43.650592,-79.385806,"[{'label': 'display', 'lat': 43.650592, 'lng':...",391,CA,Toronto,ON,Canada,"[145 Queen St. W (at University Ave.), Toronto...",145 Queen St. W,at University Ave.,M5H 4G1,NaN,4ad4c062f964a520e5f720e3
2,Art Gallery of Ontario,Art Gallery,43.654003,-79.392922,"[{'label': 'display', 'lat': 43.65400286033738...",460,CA,Toronto,ON,Canada,"[317 Dundas St W (at Beverley St), Toronto ON ...",317 Dundas St W,at Beverley St,M5T 1G4,NaN,4ad4c05ef964a520daf620e3
3,Nathan Phillips Square,Plaza,43.652270,-79.383516,"[{'label': 'display', 'lat': 43.65227047322295...",351,CA,Toronto,ON,Canada,"[100 Queen St W (at Bay St), Toronto ON M5H 2N...",100 Queen St W,at Bay St,M5H 2N1,NaN,4ad4c05ef964a520a6f620e3
4,Aboveground Art Supplies,Arts & Crafts Store,43.652646,-79.390925,"[{'label': 'display', 'lat': 43.65264564807000...",333,CA,Toronto,ON,Canada,"[74 McCaul St (Dundas St.), Toronto ON M5T 3K2...",74 McCaul St,Dundas St.,M5T 3K2,NaN,4adf3c01f964a5208f7821e3
5,Rosalinda,Vegetarian / Vegan Restaurant,43.650252,-79.385156,"[{'label': 'display', 'lat': 43.65025248306929...",444,CA,Toronto,ON,Canada,"[133 Richmond St W, Toronto ON M5H 2L2, Canada]",133 Richmond St W,NaN,M5H 2L2,NaN,5aff06ca6e4650002cc6286b
6,UNIQLO ユニクロ,Clothing Store,43.655910,-79.380641,"[{'label': 'display', 'lat': 43.65591027779457...",571,CA,Toronto,ON,Canada,"[220 Yonge St (at Dundas St W), Toronto ON M5B...",220 Yonge St,at Dundas St W,M5B 2H1,Downtown Toronto,57eda381498ebe0e6ef40972
7,Shangri-La Toronto,Hotel,43.649129,-79.386557,"[{'label': 'display', 'lat': 43.64912919417502...",540,CA,Toronto,ON,Canada,"[188 University Ave. (at Adelaide St. W), Toro...",188 University Ave.,at Adelaide St. W,M5H 0A3,NaN,4e31b74252b131dcebb08743
8,The Keg Steakhouse + Bar,Steakhouse,43.649937,-79.384196,"[{'label': 'display', 'lat': 43.64993725298525...",509,CA,Toronto,ON,Canada,[165 York St (btwn Richmond St. & Adelaide St....,165 York St,btwn Richmond St. & Adelaide St.,M5H 3R8,NaN,4ad69511f964a520e40721e3
9,Hailed Coffee,Coffee Shop,43.658833,-79.383684,"[{'label': 'display', 'lat': 43.65883296982352...",611,CA,Toronto,ON,Canada,"[44 Gerrard St W (Yonge St), Toronto ON M5G, C...",44 Gerrard St W,Yonge St,M5G,College Park,5a81ae339deb7d369fa7f146


In [40]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map 


# add Cith centre as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular venues to the map as blue circle markers
for lat, lng, label in zip(df_filtered.lat, df_filtered.lng, df_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

In [69]:
## create location matrix to perform K-Means

X=df_filtered[['lat','lng']]
X

,lat,lng
0,43.653232,-79.385296
1,43.650592,-79.385806
2,43.654003,-79.392922
3,43.652270,-79.383516
4,43.652646,-79.390925
...,...,...
95,43.645119,-79.395883
96,43.642536,-79.387182
97,43.650063,-79.374587
98,43.651481,-79.373600


In [73]:
X=df_filtered.iloc[:,2:4]
X

,lat,lng
0,43.653232,-79.385296
1,43.650592,-79.385806
2,43.654003,-79.392922
3,43.652270,-79.383516
4,43.652646,-79.390925
...,...,...
95,43.645119,-79.395883
96,43.642536,-79.387182
97,43.650063,-79.374587
98,43.651481,-79.373600


In [74]:
from sklearn.cluster import KMeans 
k_means = KMeans(init="k-means++", n_clusters=4, n_init=12)   # we divide the venues into 4 parts, which means 4 sub-centres.
k_means.fit(X)


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=4, n_init=12, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [75]:
## get labels
k_means_labels = k_means.labels_

In [76]:
k_means_cluster_centers = k_means.cluster_centers_
k_means_cluster_centers

array([[ 43.64971767, -79.39684474],
       [ 43.64720391, -79.3863034 ],
       [ 43.65857771, -79.38108579],
       [ 43.6507256 , -79.3772266 ]])

In [87]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)
import matplotlib.cm as cm
import matplotlib.colors as colors
# set color scheme for the clusters
x = np.arange(4)
ys = [i + x + (i*x)**2 for i in range(4)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, cluster in zip(X.lat, X.lng ,k_means_labels ):
    
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
        
##add center to the map        
k=4
m=[2*(i*k)**2 for i in range(4)]
color=cm.rainbow(np.linspace(0,2,len(ys)))
c2=[colors.rgb2hex(i) for i in colors_array]        
for i,cluster in zip(k_means_cluster_centers,range(4)):
        folium.CircleMarker(
        i,
        radius=10,
        
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=1).add_to(map_clusters) 
        
map_clusters 

## we need to explore if there are to many compatitors nearby by visualizing them.

In [118]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel



# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
    
k=4
m=[2*(i*k)**2 for i in range(4)]
color=cm.rainbow(np.linspace(0,2,len(ys)))
c2=[colors.rgb2hex(i) for i in colors_array]        
for i,cluster in zip(k_means_cluster_centers,range(4)):
        folium.CircleMarker(
        i,
        radius=10,
        
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=1).add_to(venues_map) 

        
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)
        
venues_map
        
        

## Summarize the number of venues near four centres and the distance between them and city center.

In [99]:
a0=sum(k_means_labels==0)
a1=sum(k_means_labels==1)
a2=sum(k_means_labels==2)
a3=sum(k_means_labels==3)
print(a0,a1,a2,a3)

33 29 15 23


In [109]:
##the distance between them and city center

d1=(k_means_cluster_centers[0][0]-latitude)**2+(k_means_cluster_centers[0][1]-longitude)**2
d2=(k_means_cluster_centers[1][0]-latitude)**2+(k_means_cluster_centers[1][1]-longitude)**2
d3=(k_means_cluster_centers[2][0]-latitude)**2+(k_means_cluster_centers[2][1]-longitude)**2
d4=(k_means_cluster_centers[3][0]-latitude)**2+(k_means_cluster_centers[3][1]-longitude)**2
print(d1,d2,d3,d4)

0.00011090881367561433 4.6501819270064964e-05 5.876476549972488e-05 0.00011008910843907614
